In [1]:
import random
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup,BertConfig

In [2]:
from config import parse_args

args = parse_args()
def setup_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
setup_seed(args.seed)

In [3]:
data_path = '../dataset/train.csv'
df = pd.read_csv(data_path, delimiter="\t")
df['tag'] = df['tag'].apply(lambda x: eval(x))
df.info()

df.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6000 non-null   object
 1   tag     6000 non-null   object
dtypes: object(2)
memory usage: 93.9+ KB


,text,tag
0,会安博物馆等，漫步会安古镇各精致的工艺品店、品尝路边的小吃摊，体验当地的风土民情。,[会安古镇]
1,贝蒂斯vs西班牙人,"[贝蒂斯, 西班牙人]"
2,最终橘子熊在特种部队项目以7：2，跑跑卡丁车项目以7：1痛击曜越太阳神，,[橘子熊]
3,2008年11月22日，北京的气温陡降到零下4度，但雍和宫星光现场里“beijing,[北京]
4,光谱代理《大战略PERFECT3》繁体版,[光谱]


In [4]:
bio_list = []
for i in tqdm(range(len(df))):
    text = df['text'][i]
    tags = df['tag'][i]
    bios = ['O']*len(text)
    for t in tags:
        idx = text.find(t)
        bios[idx] = 'B-0'
        for j in range(idx+1, idx+len(t)):
            bios[j] = 'I-0'
    bio_list.append(bios)

100%|██████████| 6000/6000 [00:00<00:00, 120001.83it/s]


In [5]:
bio_list = [' '.join(i) for i in bio_list]

In [6]:
df['bio'] = bio_list


In [7]:
from sklearn.model_selection import train_test_split
train_data, valid_data = train_test_split(df, test_size = 0.2, random_state=args.seed)
train_data.index = list(range(len(train_data)))
valid_data.index = list(range(len(valid_data)))
# print(len(train_data), len(valid_data))
print('训练集大小：',len(train_data))
print('验证集大小：',len(valid_data))

训练集大小： 4800
验证集大小： 1200


In [8]:

args.tag2idx = {'O':0, 'B-0':1, 'I-0':2}
args.idx2tag = {0: 'O', 1: 'B-0', 2:'I-0'}

In [9]:

print('pretrained save_model use'+args.bert_dir)
tokenizer = BertTokenizer.from_pretrained(args.bert_dir)

from data_helper import create_data_loader
train_data_loader = create_data_loader(train_data['text'], train_data['bio'], args, tokenizer)
valid_data_loader = create_data_loader(valid_data['text'], valid_data['bio'], args, tokenizer)

pretrained save_model use../pretrain_models/hfl/chinese-roberta-wwm-ext


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
F:\ML_ENVS\lib\site-packages\transformers\tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
print(len(train_data_loader), len(valid_data_loader))

300 75


In [11]:
def jaccard_score(pred, label):
    return len(set(pred) & set(label)) / len(set(pred) | set(label))

In [12]:
def train_epoch(model, data_loader, optimizer, args, scheduler):
    # 训练模式
    model = model.train()
    train_loss = 0
    for sample in tqdm(data_loader):
        input_ids = sample['input_ids'].to(args.device)
        attention_mask = sample['attention_mask'].to(args.device)
        label_ids = sample['label_ids'].to(args.device)
        out, loss = model(input_ids=input_ids,
                        label_ids=label_ids,
                        attention_mask=attention_mask)

        train_loss += loss.item()
        loss.backward()

        # -----------------------------------对抗攻击------------------------------------------------
        if args.use_fgm:
            # 对抗训练
            fgm.attack()  # 在embedding上添加对抗扰动
            # loss_adv = model(batch_input, batch_label)
            out, loss_adv = model(input_ids=input_ids,
                        label_ids=label_ids,
                        attention_mask=attention_mask)
            loss_adv = loss_adv.mean()
            loss_adv.backward()  # 反向传播，并在正常的grad基础上，累加对抗训练的梯度
            fgm.restore()  # 恢复embedding参数

        if args.use_pgd:
            pgd.backup_grad()
            for t in range(K):
                pgd.attack(is_first_attack=(t == 0))
                if t != K - 1:
                    model.zero_grad()
                else:
                    pgd.restore_grad()

                out, loss_adv = model(input_ids=input_ids,
                                      label_ids=label_ids,
                                      attention_mask=attention_mask)
                loss_adv = loss_adv.mean()
                loss_adv.backward()  # 反向传播，并在正常的grad基础上，累加对抗训练的梯度

            pgd.restore()


            # ----------------------------------------------------------------------------------------




        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        optimizer.zero_grad()
        scheduler.step()

        if args.ema != False:
            args.ema.update()



    return train_loss/len(data_loader)

from ark_nlp.factory.utils.conlleval import get_entity_bio
def return_entity(label):
    entity_labels = []
    for _type, _start_idx, _end_idx in get_entity_bio(label, id2label=None):
            entity_labels.append({
                'start_idx': _start_idx,
                'end_idx': _end_idx,
                'type': _type
            })
    entity_labels = [str(dic['start_idx'])+'-'+str(dic['end_idx']) for dic in entity_labels]
    return entity_labels


def eval_epoch(model, data_loader, args):
    # 验证模式
    model = model.eval()
    if args.ema!=False:
        args.ema.apply_shadow()
    val_loss = 0
    jc_score_list = []
    # 关闭自动求导，省内存加速，因为是不是训练模式了，没必要求导
    with torch.no_grad():
        for sample in tqdm(data_loader):
            input_ids = sample['input_ids'].to(args.device)
            attention_mask = sample['attention_mask'].to(args.device)
            label_ids = sample['label_ids'].to(args.device)
            out, loss = model(input_ids=input_ids,
                        label_ids=label_ids,
                        attention_mask=attention_mask)

            val_loss += loss.item()


            predict_ids = out
            # predict_ids
            #%%
            label_ids = sample['label_ids'].numpy().tolist()

            entity_all_label_ids = []
            entity_all_predict_ids = []
            for i in range(len(label_ids)):
                tmp_label, tmp_predict = [], []
                # 因为我crf有做mask所以这里的len(len(predict_tag[i]))是不带有pad的长度
                for j in range(0, len(predict_ids[i])):
                    tmp_label.append(args.idx2tag[label_ids[i][j]])
                    tmp_predict.append(args.idx2tag[predict_ids[i][j]])
                entity_all_label_ids.append(tmp_label)
                entity_all_predict_ids.append(tmp_predict)


            for label, pred in zip(entity_all_label_ids, entity_all_predict_ids):
                label_entity = return_entity(label)
                pred_entity = return_entity(pred)
                jc_score_list.append(jaccard_score(pred=pred_entity, label=label_entity))

    return val_loss/len(data_loader), np.mean(jc_score_list)


In [13]:
from model import BERT_CRF

if torch.cuda.is_available():
    args.device = 'cuda:0'
    print('使用：', args.device,' ing........')

model = BERT_CRF(args=args).to(args.device)


print('batch_size: ',args.batch_size, 'epochs: ',args.max_epochs)
num_total_steps = len(train_data_loader) * args.max_epochs
from util import build_optimizer, build_optimizer_diff_lr
optimizer, scheduler = build_optimizer(args, model, num_total_steps=num_total_steps)
# optimizer, scheduler = build_optimizer_diff_lr(args, model, num_total_steps=num_total_steps)



if args.ema==True:
    print('-'*10,'采用EMA机制训练','-'*10)
    from tricks import EMA
    args.ema = EMA(model, 0.999)
    args.ema.register()

if args.use_fgm==True:
    print('-' * 10, '采用FGM对抗训练', '-' * 10)
    from tricks import FGM
    # 初始化
    fgm = FGM(model)

if args.use_pgd==True:
    print('-' * 10, '采用PGD对抗训练', '-' * 10)
    from tricks import PGD
    # 初始化
    pgd = PGD(model=model)
    K = 3

使用： cuda:0  ing........
batch_size:  16 epochs:  6
learning_rate:  5e-05
num_training_steps:  1800
warmup_steps:  108.0


Some weights of the model checkpoint at ../pretrain_models/hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
best_jc_score = 0
for epoch in range(args.max_epochs):
    print('——'*10, f'Epoch {epoch + 1}/{args.max_epochs}', '——'*10)
    train_loss = train_epoch(model, train_data_loader, optimizer, args, scheduler)
    # #scheduler.step()
    # print('-'*20)
    print(f'Train loss : {round(train_loss, 2)}\n')
    val_loss, jc_score = eval_epoch(model, valid_data_loader, args)



    if jc_score>best_jc_score:
        best_jc_score = jc_score
        print(f'val loss : {round(val_loss, 3)}')
        print(f"jc_score: {round(jc_score, 3)}")
        print('-'*20)
        torch.save(model.state_dict(), './save_model/best_model.pth')
        print('+'*6,'best save_model saved','+'*6)

    if args.ema != False:
        args.ema.restore()

———————————————————— Epoch 1/6 ————————————————————
Train loss : 146.45

val loss : 74.172
jc_score: 0.727
--------------------
++++++ best save_model saved ++++++
———————————————————— Epoch 2/6 ————————————————————
Train loss : 54.5

val loss : 77.201
jc_score: 0.741
--------------------
++++++ best save_model saved ++++++
———————————————————— Epoch 3/6 ————————————————————
Train loss : 33.92

val loss : 86.252
jc_score: 0.778
--------------------
++++++ best save_model saved ++++++
———————————————————— Epoch 4/6 ————————————————————
Train loss : 22.57

val loss : 112.597
jc_score: 0.784
--------------------
++++++ best save_model saved ++++++
———————————————————— Epoch 5/6 ————————————————————
Train loss : 16.35

val loss : 134.848
jc_score: 0.784
--------------------
++++++ best save_model saved ++++++
———————————————————— Epoch 6/6 ————————————————————
Traceback (most recent call last):
  File "F:\ML_ENVS\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
 

 23%|██▎       | 68/300 [00:12<00:44,  5.25it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



TypeError: object of type 'NoneType' has no len()